In [ ]:
import numpy as np
import pandas as pd
from pyspark.sql.types import IntegerType
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession

In [ ]:
# Create spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

#    .config( \
#    "spark.archives",  \
#    "pyspark_venv.tar.gz#environment") \

# Read in full sparkify dataset
feature_data = "./features.json"

df = spark.read.json(event_data)
df.persist()
df.createOrReplaceTempView('df')
df.head()